<a href="https://colab.research.google.com/github/DanielSchussmann/Seminar/blob/main/Seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $$\text{Machine Learning updates} $$


##$$\text{ Variables}$$ 

$$ i = \text{parameter for the layer}$$
$$ j,l= \text{parameter for the position inside the i-th layer}$$
$$ b_{i,j} = \text{Bias of neuron} $$

$$ x_{i,j} = \text{input value / fireing rate}$$
$$ w_{i,j,l} = \text{weight of connection, k and l are destination neuron}$$
$$\sigma = \text{activation function}$$
$$n_i = \text{number of neurons in the i-th layer}$$

(add illustration for clarification)

$$x_{i+1,j} = \sigma [\sum_{j=1}^{n_i}(w_{(i,j,l)}^T * x_{i,j}) +b_{i+1,j}] \text{(not a recursive expression)}$$
$$ .\\.\\.\\ x_{1,1} = \sigma [\sum_{j=1}^{n_0}(w_{(0,j,1)}^T * x_{0,j}) +b_{1,1}] \\ $$
$$ x_{2,4} = \sigma [\sum_{j=1}^{n_1}(w_{(1,j,4)}^T * x_{1,j}) +b_{2,4}] \\ $$

In [51]:
#WHY USE A FRAMEWORK LIKE TENSORFLOW
import tensorflow as tf

raw_data = [5.48813504, 7.15189366, 6.02763376, 5.44883183, 4.23654799, 6.45894113, 4.37587211, 8.91773001, 9.63662761, 3.83441519]


#--Normalizing-without-any-libraries-------------------------------

def normalize(inp):
    A = [i*i for i in inp]
    v = (sum(A))**(0.5)
    A = [i / v for i in inp]
    return A


print(normalize(raw_data))


#--Normalizing-with-Tensorflow-------------------------------------


print(tf.keras.utils.normalize(raw_data))




[0.2699883994478898, 0.3518368823309622, 0.296528999279228, 0.26805488092407004, 0.20841666662128538, 0.3177471336085864, 0.2152707064525075, 0.4387070716299857, 0.4740731861618412, 0.18863377311626361]
[[0.2699884  0.35183688 0.296529   0.26805488 0.20841667 0.31774713
  0.21527071 0.43870707 0.47407319 0.18863377]]
